In [1]:
import pandas as pd
import numpy as np
import math
import os
import tensorflow as tf
from tensorflow import keras
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt 
import seaborn as sns            # visualization tool
import matplotlib.cm as cm       #

In [2]:
def zero_crossing(temp,threshold):
    zc = 0
    for i in range(len(temp)-1):
        if((temp[i]*temp[i+1]) < 0 and abs(temp[i]-temp[i+1]) >= threshold):
            zc += 1
    return zc

In [3]:
def process_one_csv(file_name,k):
    data = pd.read_csv('/home/amaya/emp/Realtime-Sign-Language-Translation-to-Speech-DNN/all/User'+str(k)+'/'+file_name)
    global classes
    
    class_ = file_name[0:len(file_name)-14]
    
    # print(data.head(2))

    columns = list(data.columns)[1:]
    # features = ['MAV','RMS','VAR','SSI','MAX','MIN']
    
    vector = []
    # global out_columns
    
    for item in columns:
        temp = list(data[item])
        # calculating MAV ***
        # out_columns.append(item+features[0])
        abs_val = list(map(abs,temp))
        vector.append(np.mean(abs_val))

        # calculating RMS ***
        # out_columns.append(item+features[1])
        vector.append(np.sqrt(np.mean(np.array(temp)**2)))

        # calculate variance ***
        # out_columns.append(item+features[2])
        x_mean = np.mean(temp)
        dif = temp - x_mean
        vector.append(np.mean(np.array(dif)**2))

        # calculating ssi ***
        # out_columns.append(item+features[3])
        vector.append(np.sum(np.array(temp)**2))

        # calculating max ***
        # out_columns.append(item+features[4])
        vector.append(max(temp))

        # calculating min ***
        # out_columns.append(item+features[5])
        vector.append(min(temp))
        
        # calculating zero crossing
        if((item[0] == 'E') or (item[0] == 'G')):
            vector.append(zero_crossing(temp,1))
        elif(item[0] == 'A'):
            vector.append(zero_crossing(temp,0.15))
        
    if class_ in classes:
        vector.append(classes.index(class_))
    else:
        classes.append(class_)
        vector.append(classes.index(class_))
        
    return vector



In [4]:
out_columns = ['EMG0LMAV', 'EMG0LRMS', 'EMG0LVAR', 'EMG0LSSI', 'EMG0LMAX', 'EMG0LMIN','EMG0ZC', 'EMG1LMAV', 'EMG1LRMS', 'EMG1LVAR', 'EMG1LSSI', 'EMG1LMAX', 'EMG1LMIN','EMG1ZC', 'EMG2LMAV', 'EMG2LRMS', 'EMG2LVAR', 'EMG2LSSI', 'EMG2LMAX', 'EMG2LMIN', 'EMG2ZC','EMG3LMAV', 'EMG3LRMS', 'EMG3LVAR', 'EMG3LSSI', 'EMG3LMAX', 'EMG3LMIN', 'EMG3ZC','EMG4LMAV', 'EMG4LRMS', 'EMG4LVAR', 'EMG4LSSI', 'EMG4LMAX', 'EMG4LMIN', 'EMG4ZC','EMG5LMAV', 'EMG5LRMS', 'EMG5LVAR', 'EMG5LSSI', 'EMG5LMAX', 'EMG5LMIN', 'EMG5ZC','EMG6LMAV','EMG6LRMS', 'EMG6LVAR', 'EMG6LSSI', 'EMG6LMAX', 'EMG6LMIN', 'EMG6ZC','EMG7LMAV', 'EMG7LRMS', 'EMG7LVAR', 'EMG7LSSI', 'EMG7LMAX', 'EMG7LMIN', 'EMG7ZC','AXLMAV', 'AXLRMS', 'AXLVAR', 'AXLSSI', 'AXLMAX', 'AXLMIN', 'AXLZC','AYLMAV', 'AYLRMS', 'AYLVAR', 'AYLSSI', 'AYLMAX', 'AYLMIN', 'AYLZC','AZLMAV', 'AZLRMS', 'AZLVAR', 'AZLSSI', 'AZLMAX', 'AZLMIN', 'AZLZC','GXLMAV', 'GXLRMS', 'GXLVAR', 'GXLSSI', 'GXLMAX', 'GXLMIN', 'GXLZC','GYLMAV', 'GYLRMS', 'GYLVAR', 'GYLSSI', 'GYLMAX', 'GYLMIN', 'GYLZC','GZLMAV', 'GZLRMS', 'GZLVAR', 'GZLSSI', 'GZLMAX', 'GZLMIN', 'GZLZC','ORLMAV', 'ORLRMS', 'ORLVAR', 'ORLSSI', 'ORLMAX', 'ORLMIN', 'OPLMAV', 'OPLRMS', 'OPLVAR', 'OPLSSI', 'OPLMAX', 'OPLMIN', 'OYLMAV', 'OYLRMS', 'OYLVAR', 'OYLSSI', 'OYLMAX', 'OYLMIN', 'EMG0RMAV', 'EMG0RRMS', 'EMG0RVAR', 'EMG0RSSI', 'EMG0RMAX', 'EMG0RMIN', 'EMG0RZC', 'EMG1RMAV', 'EMG1RRMS', 'EMG1RVAR', 'EMG1RSSI', 'EMG1RMAX', 'EMG1RMIN','EMG1RZC', 'EMG2RMAV', 'EMG2RRMS', 'EMG2RVAR', 'EMG2RSSI', 'EMG2RMAX', 'EMG2RMIN','EMG2RZC', 'EMG3RMAV', 'EMG3RRMS', 'EMG3RVAR', 'EMG3RSSI', 'EMG3RMAX', 'EMG3RMIN','EMG3RZC', 'EMG4RMAV', 'EMG4RRMS', 'EMG4RVAR', 'EMG4RSSI', 'EMG4RMAX', 'EMG4RMIN','EMG4RZC', 'EMG5RMAV', 'EMG5RRMS', 'EMG5RVAR', 'EMG5RSSI', 'EMG5RMAX', 'EMG5RMIN','EMG5RZC', 'EMG6RMAV', 'EMG6RRMS', 'EMG6RVAR', 'EMG6RSSI', 'EMG6RMAX', 'EMG6RMIN','EMG6RZC', 'EMG7RMAV', 'EMG7RRMS', 'EMG7RVAR', 'EMG7RSSI', 'EMG7RMAX', 'EMG7RMIN','EMG7RZC', 'AXRMAV', 'AXRRMS', 'AXRVAR', 'AXRSSI', 'AXRMAX', 'AXRMIN','AXRZC', 'AYRMAV', 'AYRRMS', 'AYRVAR', 'AYRSSI', 'AYRMAX', 'AYRMIN','AYRZC', 'AZRMAV', 'AZRRMS', 'AZRVAR', 'AZRSSI', 'AZRMAX', 'AZRMIN','AZRZC', 'GXRMAV', 'GXRRMS', 'GXRVAR', 'GXRSSI', 'GXRMAX', 'GXRMIN','GXRZC', 'GYRMAV', 'GYRRMS', 'GYRVAR', 'GYRSSI', 'GYRMAX', 'GYRMIN','GYRZC', 'GZRMAV', 'GZRRMS', 'GZRVAR', 'GZRSSI', 'GZRMAX', 'GZRMIN','GZRZC', 'ORRMAV', 'ORRRMS', 'ORRVAR', 'ORRSSI', 'ORRMAX', 'ORRMIN', 'OPRMAV', 'OPRRMS', 'OPRVAR', 'OPRSSI', 'OPRMAX', 'OPRMIN', 'OYRMAV', 'OYRRMS', 'OYRVAR', 'OYRSSI', 'OYRMAX', 'OYRMIN','CLASS']
# 205 columns (34*6 + 1)
output = []
classes=[]
# print(len(out_columns))

# process_one_csv(files[0])
for k in range(1,11):
    if(k!=5):
        files = os.listdir('/home/amaya/emp/Realtime-Sign-Language-Translation-to-Speech-DNN/all/User'+str(k)+'/')
    else:
        continue
    for file in files:
        if(file[-1] == 'v'):      # to avoid _DS_STORE files and anyother hiiden files
            output.append(process_one_csv(file,k))
        
processed = pd.DataFrame(output,columns = out_columns)
processed.shape

(550, 233)

In [5]:
# feature scaling of data
scaler = preprocessing.MaxAbsScaler()   # since the data set is not gaussian
scaled_df = scaler.fit_transform(processed[out_columns[:len(out_columns)-1]])
X = pd.DataFrame(scaled_df, columns = out_columns[:len(out_columns)-1])
y = processed['CLASS']

In [6]:
# splitting the dataset
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.20)

In [7]:
# setting up the layers
model = keras.Sequential([
    keras.layers.Dense(232,input_dim = 232,activation = 'relu'),
    keras.layers.Dense(450,activation = 'relu'),
    keras.layers.Dense(20,activation = 'softmax')
])

# compile the model
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

W0823 16:23:43.267088 140430451128064 deprecation.py:506] From /home/amaya/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [8]:
# Training the model
model.fit(X_train,y_train,epochs = 100)

Epoch 1/100
440/440 [==============================] - 0s 285us/sample - loss: 2.8761 - acc: 0.1500
Epoch 2/100
440/440 [==============================] - 0s 79us/sample - loss: 2.3780 - acc: 0.3500
Epoch 3/100
440/440 [==============================] - 0s 76us/sample - loss: 1.9216 - acc: 0.4295
Epoch 4/100
440/440 [==============================] - 0s 78us/sample - loss: 1.6148 - acc: 0.5159
Epoch 5/100
440/440 [==============================] - 0s 88us/sample - loss: 1.3310 - acc: 0.6182
Epoch 6/100
440/440 [==============================] - 0s 74us/sample - loss: 1.1600 - acc: 0.6591
Epoch 7/100
440/440 [==============================] - 0s 76us/sample - loss: 1.0086 - acc: 0.7136
Epoch 8/100
440/440 [==============================] - 0s 74us/sample - loss: 0.8657 - acc: 0.7795
Epoch 9/100
440/440 [==============================] - 0s 77us/sample - loss: 0.7349 - acc: 0.8114
Epoch 10/100
440/440 [==============================] - 0s 75us/sample - loss: 0.6470 - acc: 0.8227
Epoch 11

In [9]:
# evaluate accuracy
test_loss,test_acc = model.evaluate(X_test,y_test)
print(test_acc)

110/110 [==============================] - 0s 387us/sample - loss: 0.3310 - acc: 0.9182
0.91818184


In [10]:
# predict
predictions = model.predict(X_test)
for i in range(20):
    print(np.argmax(predictions[i]))

19
11
13
0
9
16
17
18
1
17
0
18
19
11
4
17
0
7
2
1


In [11]:
for i in range(20):
    print(list(y_test)[i])

19
11
13
0
9
16
17
18
1
17
15
18
19
11
4
17
0
7
2
1
